<a href="https://colab.research.google.com/github/arijitar/Diabetic-Retinopathy-detection/blob/main/Optimized_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Loading all the neccessary libraries as per the requirement i.e.
# 1. Reading the files = os
# 2. Image Processing  = cv2
# 3. Deep Learning  = tensorflow
# 4. Splitting the data & Endcoding the Labels = sklearn
# 5. For cleaning memory = gc
!pip install tensorflow
!pip install opencv-python
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2DTranspose, Conv2D, LeakyReLU, MaxPooling2D, Dropout, concatenate, GlobalAveragePooling2D, Cropping2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_size = 224 # By deafult image size is (299,299,3) but due to limited RAM and GPU provided by the google colab it crashes.
# Therefore to overcome the problem of GPU crashes we have reduced the size of an image i.e. (150,150,3)

# path = '/content/drive/MyDrive/DRD_new/drdataset/dataset/Training' # It points to the original dataset path
path = '/content/drive/MyDrive/Aug_Dataset1' # It points to the augmented dataset path

image_type_folders = os.listdir(path) # stores the entire path to the dataset into this variable

print("Folders inside Dataset Folder = ",image_type_folders)

images = [] # Images list will store all the images
labels = [] # labels list will store "positve" & "negative" labels

for folder in image_type_folders:
    folder_path = os.path.join(path, folder)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (image_size, image_size)) # resize all the images
        images.append(img) # append the images into the images list
        labels.append(folder.strip().lower()) # append the folder names as labels i.e. positive & negative inside the labels list

In [ ]:
images = np.array(images).astype('float32') / 255.0 # Converts the image list into numpy array and normalize all the pixel values into the range of (0-1)
labels = np.array(labels) # Make labels a Numpy array for easier processing

In [ ]:
images = np.load('/content/drive/MyDrive/preprocessed_images.npy')
labels = np.load('/content/drive/MyDrive/preprocessed_labels.npy')

In [ ]:
# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)
Y = to_categorical(labels)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, Y, test_size=0.1, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1.0)

train_generator = datagen.flow(x_train, y_train, batch_size=64, shuffle=True)
val_generator = datagen.flow(x_val, y_val, batch_size=64, shuffle=False)
test_generator = datagen.flow(x_test, y_test, batch_size=64, shuffle=False)


efficient net implementation

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Sequential, Model

numberOfClasses = 2
imgSize = 224
size = (imgSize, imgSize)

inputs = layers.Input(shape=(imgSize, imgSize, 3))

#Creating our custom EfficientNet model that's why weights = None because we don't want to use pre-defined imagenet weights & include_top = True
outputs = EfficientNetB0(include_top=True, weights=None, classes=numberOfClasses)(inputs)



In [ ]:
Enet_model = Model(inputs, outputs)

In [ ]:
Enet_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"] )

In [ ]:
Enet_model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

In [ ]:
history = Enet_model.fit(x_train, y_train, epochs=24, batch_size = 32, validation_split=0.2, callbacks=[early_stop])

In [ ]:
y_predict = Enet_model.predict(x_test)
y_predict

In [ ]:
import math
loss, test_accuracy = Enet_model.evaluate(x_test, y_test) # Accuracy score is 92.37%
accuracy = test_accuracy * 100
# print(f"Overall Accuracy of the Model = {accuracy:.3f}%")
print(accuracy)
rounded = math.ceil(accuracy)
print(f"Overall Accuracy of the model = {rounded}%")

In [ ]:
import matplotlib.pyplot as plt

x_value = test_accuracy

def plot_accuracyGraph(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.history['accuracy'], color = 'orange', linewidth = 1, marker = '.')
  plt.plot(history.history['val_accuracy'], color='blue', linewidth=1, marker='.', label='Validation Accuracy')
  plt.plot([x_value,x_value], color='green', linestyle='dashed', label=f'x = {x_value}', linewidth = 1, marker = '.')
  plt.title('Train,Test & Validation Accuracy Graph')
  plt.ylabel('Accuracy & Validation Accuracy')
  plt.xlabel('Epoch')
  plt.grid(True)
  plt.legend(['Train', 'Validation', 'Test'], loc='lower center')
  plt.show()

plot_accuracyGraph(history)

In [ ]:
Enet_model.save('/content/drive/MyDrive/Optimized_EfficientNet_model.keras')  # TensorFlow SavedModel format (recommended)